In [ ]:
from IPython.core.display import HTML
with open ("../style.css", "r") as file:
    css = file.read()
HTML(css)

# A Grammar for Propositional Logic

This file shows how a simple symbolic calculator can be implemented using `Ply`.  The grammar for the language implemented by this parser is as follows:
$$
\begin{array}{lcl}
  \texttt{stmnt}   & \rightarrow & \;\texttt{IDENTIFIER} \;\texttt{':='}\; \texttt{formula}\; \texttt{';'}\\
                   & \mid        & \;\texttt{formula}\; \texttt{';'}                                      \\[0.2cm]
  \texttt{formula} & \rightarrow & \;\texttt{formula}\; \texttt{'<->'} \; \texttt{formula}                \\
                   & \mid        & \;\texttt{formula}\; \texttt{'->'} \; \texttt{formula}                 \\
                   & \mid        & \;\texttt{formula}\; \texttt{'&'} \; \texttt{formula}                  \\
                   & \mid        & \;\texttt{'!'} \;\texttt{formula}                                      \\
                   & \mid        & \;\texttt{'('} \; \texttt{formula} \;\texttt{')'}                      \\
                   & \mid        & \;\texttt{VALUE}                                                       \\
                   & \mid        & \;\texttt{IDENTIFIER}                        
\end{array}
$$

## Specification of the Scanner

In [ ]:
import ply.lex as lex

There are only five tokens that need to be defined via regular expressions.  The other tokens consist only of a single character and are therefore 
defined as literals.

In [ ]:
tokens = [ 'VALUE', 'IDENTIFIER', 'ASSIGN_OP', 'BICONDITIONAL', 'CONDITIONAL' ]

The token `VALUE` specifies a *truth value*.  We support the strings `True`and `False` as truth values.  
Furthermore, `0` and `1` can be used as truth values.

As the strings `True` and `False` have the same structure that is specified by the regular expression for the token `IDENTIFIER`, it is <b>important</b> that the definition of `VALUE` precedes the definition of `IDENTIFIER`.

Furthermore, the string `'0'` has to be converted into the integer `0` first before this integer is then converted into a Boolean.

In [ ]:
def t_VALUE(t):
    r'True|False|0|1'
    if t.value in ['True', 'False']:
        t.value = bool(t.value)
    else:
        t.value = bool(int(t.value))
    return t

The token `IDENTIFIER` specifies the name of a *variable*.

In [ ]:
def t_IDENTIFIER(t):
    r'[a-zA-Z][a-zA-Z0-9_]*'
    return t

The token `ASSIGN_OP` specifies the *assignment operator*.  As this operator consists of two characters, it can't be defined as a literal.

In [ ]:
def t_ASSIGN_OP(t):
    r':='
    return t

def t_BICONDITIONAL(t):
    r'<->'
    return t

def t_CONDITIONAL(t):
    r'->'
    return t

`literals` is a list operator symbols that consist of a single character.

In [ ]:
literals = ['|', '&', '!', '(', ')', ';']

Blanks and tabulators are ignored.

In [ ]:
t_ignore  = ' \t'

Newlines are counted in order to give precise error messages.  Otherwise they are ignored.

In [ ]:
def t_newline(t):
    r'\n+'
    t.lexer.lineno += t.value.count('\n')

Unkown characters are reported as lexical errors.

In [ ]:
def t_error(t):
    print(f"Illegal character '{t.value[0]}' at character number {t.lexer.lexpos} in line {t.lexer.lineno}.")
    t.lexer.skip(1)

In [ ]:
__file__ = 'main'

We generate the lexer.

In [ ]:
lexer = lex.lex()

## Specification of the Parser

In [ ]:
import ply.yacc as yacc

The *start variable* of our grammar is `statement`.

In [ ]:
start = 'stmnt'

There are two grammar rules for `stmnt`s:
```
    stmnt : IDENTIFIER ":=" formula ";"
          | formula ';'
          ;
```
- If a *stmnt* is an assignment, the expression on the right hand side of the assignment operator is 
  evaluated and the value is stored in the dictionary `Names2Values`.  The key used in this dictionary
  is the name of the variable on the left hand side ofthe assignment operator.
- If a *stmnt* is a propositional formula, the formula is evaluated and the result of this evaluation is printed.


Below, `Names2Values` is a dictionary mapping variable names to their values.  It will be defined later.

In [ ]:
def p_stmnt_assign(p):
    "stmnt : IDENTIFIER ASSIGN_OP formula ';'"
    Names2Values[p[1]] = p[3]

def p_stmnt_formula(p):
    "stmnt : formula ';'"
    print(p[1])

The grammar rules for `formula` are:
```
 formula : formula '<->' formula
         | formula '->' formula
         | formula '|' formula
         | formula '&' formula
         | '!' formula
         | '(' formula ')'
         | VALUE
         | IDENTIFIER
         ;
```

In [ ]:
def p_formula_equivalence(p):
    "formula : formula BICONDITIONAL formula"
    p[0] = (p[1] == p[3])

def p_formula_implication(p):
    "formula : formula CONDITIONAL formula"
    p[0] = not p[1] or p[3]

def p_formula_or(p):
    "formula : formula '|' formula"
    p[0] = p[1] or p[3]
    
def p_formula_and(p):
    "formula : formula '&' formula"
    p[0] = p[1] and p[3]
    
def p_formula_not(p):
    "formula : '!' formula"
    p[0] = not p[1]
    
def p_formula_paren(p):
    "formula : '(' formula ')'"
    p[0] = p[2]
    
def p_formula_value(p):
    "formula : VALUE"
    p[0] = p[1]
    
def p_formula_identifier(p):
    "formula : IDENTIFIER"
    p[0] = Names2Values[p[1]]

In [ ]:
precedence = ( ('nonassoc', 'BICONDITIONAL'),
               ('right',    'CONDITIONAL'  ),
               ('left',     '|'            ),
               ('left',     '&'            ),
               ('right',    '!'            )
             )

The method `p_error` is called if a syntax error occurs.  The argument `p` is the token that could not be read.  If `p` is `None` then there is a syntax error at the end of input.

In [ ]:
def p_error(p):
    if p:
        print(f"Syntax error at character number {p.lexer.lexpos} at token '{p.value}' in line {p.lexer.lineno}.")
    else:
        print('Syntax error at end of input.')

Setting the optional argument `write_tables` to `False` <B style="color:red">is required</B> to prevent an *obscure bug* where the parser generator tries to read an empty parse table.
We set `debug` to `True` so that the parse tables are dumped into the file `parser.out`.

In [ ]:
parser = yacc.yacc(write_tables=False, debug=True)

Let's look at the action table that is generated.

In [ ]:
!type parser.out

In [ ]:
!cat parser.out

`Names2Values` is the dictionary that maps variable names to their values.  Initially the dictionary is empty as no variables has yet been defined.

In [ ]:
Names2Values = {}

The method `test(s)` takes a string `s` that is supposed to be a `stmnt`.
This statement is then executed.

In [ ]:
def test(s):
    yacc.parse(s)

In [ ]:
test('x := True;');
test('x;')

In [ ]:
test('y := False;')

In [ ]:
test('1 <-> 0;')

In [ ]:
test('x -> y | 0 & 1 <-> (x | y);')

In [ ]:
for op in ['<->', '->', '&', '|']:
    print(f'\nTesting "{op}":')
    for x in ['0', '1']:
        for y in ['0', '1']:
            test(f'x := {x};')
            test(f'y := {y};')
            print(f'{x} {op} {y}:')
            test(f'x {op} y;')